In [1]:
#Importing the libraries
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM

In [2]:
#Importing the data
data=pd.read_table('raw_in_domain_train.tsv',header=None)

In [3]:
data

,0,1,2,3
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.
3,gj04,1,NaN,"The more we study verbs, the crazier they get."
4,gj04,1,NaN,Day by day the facts are getting murkier.
...,...,...,...,...
8546,ad03,0,*,Poseidon appears to own a dragon
8547,ad03,0,*,Digitize is my happiest memory
8548,ad03,1,NaN,It is easy to slay the Gorgon.
8549,ad03,1,NaN,I had the strangest feeling that I knew you.


In [4]:
##Transform the texts column into a list
texts=data[3].tolist()

In [5]:
##Function for cleaning texts
def clean_text(text): 
    text=re.sub("(\?|!)+"," ",text) #remvoe ? and !
    text=re.sub("\s\d+\s","",text) # remove digits 
    text=re.sub("(\.|\,)+","",text) #remove . and ,
    text=re.sub("^\s+","",text) #remove space in the begining
    text=re.sub("\s+$","",text) #remove space in the end
    
    return text

In [6]:
##Function for processing texts
def process_sentence(texts):
    clean_texts=[]
    for text in texts:
        text=clean_text(text) #Cleaning the text
        clean_texts.append(text.lower()) #Lowercasing the texts
    return clean_texts

In [7]:
clean_texts=process_sentence(texts)
clean_texts[:5]

["our friends won't buy this analysis let alone the next one we propose",
 "one more pseudo generalization and i'm giving up",
 "one more pseudo generalization or i'm giving up",
 'the more we study verbs the crazier they get',
 'day by day the facts are getting murkier']

In [8]:
#Tokenization Process and making sequence of tokens for each text
max_words = 6000 # Max size of the dictionary
tokenizer = Tokenizer(num_words=max_words) # Size of dictionary containig each word and its index
tokenizer.fit_on_texts(clean_texts) # Tokenize each word
sequences = tokenizer.texts_to_sequences(clean_texts) # convert the texts into tokenized sequences
vocab_size = len(tokenizer.word_index) # Number of unique words

In [9]:
sequences[:5],vocab_size

([[221, 271, 253, 164, 28, 698, 753, 1288, 1, 699, 79, 29, 3571],
  [79, 25, 2624, 2625, 9, 144, 1478, 49],
  [79, 25, 2624, 2625, 138, 144, 1478, 49],
  [1, 25, 29, 448, 3572, 1, 1289, 30, 210],
  [427, 34, 427, 1, 3573, 45, 902, 3574]],
 5524)

In [10]:
len(max(sequences,key=len)) #Max is a built in function to find the longest text

42

In [11]:
#Padding of the encoded texts
padded_texts=pad_sequences(sequences,maxlen=43,padding='pre')
padded_texts[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,  221,  271,  253,
         164,   28,  698,  753, 1288,    1,  699,   79,   29, 3571],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,   79,   25, 2624, 2625,    9,  144, 1478,   49],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,   79,   25, 2624, 2625,  138,  144, 1478,   49],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    

In [12]:
#Obtaining the input vector X and the label vector Y
import keras.utils as ku
label = ku.to_categorical(padded_texts[:,-1], num_classes=vocab_size)
y=label
x=padded_texts[:,:-1]
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.1)

In [18]:
model=Sequential()
model.add(Embedding(input_dim=vocab_size+1,output_dim=10,input_length=42))
model.add(LSTM(40,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 42, 10)            55250     
                                                                 
 lstm_1 (LSTM)               (None, 40)                8160      
                                                                 
 dense_2 (Dense)             (None, 100)               4100      
                                                                 
 dense_3 (Dense)             (None, 5524)              557924    
                                                                 
Total params: 625,434
Trainable params: 625,434
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train_x,train_y,epochs=100,batch_size=64)

Epoch 1/100
134/134 [==============================] - 4s 17ms/step - loss: 8.0728 - accuracy: 0.0104
Epoch 2/100
134/134 [==============================] - 2s 18ms/step - loss: 7.2124 - accuracy: 0.0112
Epoch 3/100
134/134 [==============================] - 3s 20ms/step - loss: 7.1123 - accuracy: 0.0118
Epoch 4/100
134/134 [==============================] - 3s 20ms/step - loss: 7.0494 - accuracy: 0.0123
Epoch 5/100
134/134 [==============================] - 3s 20ms/step - loss: 6.9824 - accuracy: 0.0108
Epoch 6/100
134/134 [==============================] - 3s 20ms/step - loss: 6.9006 - accuracy: 0.0098
Epoch 7/100
134/134 [==============================] - 3s 19ms/step - loss: 6.7690 - accuracy: 0.0125
Epoch 8/100
134/134 [==============================] - 3s 19ms/step - loss: 6.6108 - accuracy: 0.0163
Epoch 9/100
134/134 [==============================] - 3s 20ms/step - loss: 6.4472 - accuracy: 0.0196
Epoch 10/100
134/134 [==============================] - 3s 21ms/step - loss: 6.279

In [20]:
#Test Evaluation
model.evaluate(test_x,test_y)

27/27 [==============================] - 0s 5ms/step - loss: 0.2439 - accuracy: 0.9217


[0.24387529492378235, 0.9217289686203003]

In [21]:
#Evaluation for overfitting
model.evaluate(train_x,train_y)

241/241 [==============================] - 1s 5ms/step - loss: 0.2352 - accuracy: 0.9272


[0.23521637916564941, 0.9272254705429077]

In [24]:
#fuction to generate text including the text sent,number of words generated, model
def generate_text(text, words_generated, model):
    for i in range(words_generated):
        token_list = tokenizer.texts_to_sequences([text])[0]  #Tokenize the new text
        token_list = pad_sequences([token_list], maxlen=42, padding='pre') #padding the new token
        predicted = model.predict(token_list) #Prediction
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted.argmax(): #get the index of the max output and getting the word from the tokenizer
                output_word = word
                break
        text += " "+output_word
    return text

In [25]:
#Text Generation
generate_text('i dont remember',4,model)

1/1 [==============================] - 0s 19ms/step


'i dont remember sugar fool helen medicine'